In [1]:
import dspy
from dspy.retrieve.weaviate_rm import WeaviateRM
import weaviate
from dotenv import load_dotenv
import openai
import os

load_dotenv()
  
WCS_API_KEY = os.getenv("WCS_API_KEY")
WEAVIATE_CLUSTER_URL= os.getenv("WEAVIATE_CLUSTER_URL")
  
# Connect to a WCS instance
weaviate_client = weaviate.connect_to_wcs(
    cluster_url=WEAVIATE_CLUSTER_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WCS_API_KEY),
    headers = {
        'X-Openai-Api-Key': os.getenv("OPENAI_API_KEY")
    }
    )

llm = dspy.OpenAI(model = "gpt-4o")
retriever_model = WeaviateRM("WeaviateBlogChunk", weaviate_client=weaviate_client)

dspy.settings.configure(lm = llm, rm=retriever_model)

/Users/kevinvegda/Desktop/code/dspy-projects/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/kevinvegda/Desktop/code/dspy-projects/.venv/lib/python3.12/site-packages/google/protobuf/runtime_version.py:112: UserWarning: Protobuf gencode version 5.27.2 is older than the runtime version 5.28.1 at grpc_health/v1/health.proto. Please avoid checked-in Protobuf gencode that can be obsolete.
  warnings.warn(


In [2]:
import re

f = open("faq.md")
markdown_content = f.read()

def parse_question(markdown_content):
    question_pattern = r'#### Q: (.+?)\n'
    questions = re.findall(question_pattern, markdown_content, re.DOTALL)
    return questions

questions = parse_question(markdown_content)
questions[:5]

['Why would I use Weaviate as my vector database?',
 'What is the difference between Weaviate and for example Elasticsearch?',
 'Do you offer Weaviate as a managed service?',
 'How should I configure the size of my instance?',
 'Do I need to know about Docker (Compose) to use Weaviate?']

In [3]:
len(questions)

44

In [4]:
trainset =  questions[:20]
devset = questions[20:30]
testset = questions[30:]

trainset = [dspy.Example(question=question).with_inputs("question") for question in trainset]
devset = [dspy.Example(question=question).with_inputs("question") for question in devset]
testset = [dspy.Example(question=question).with_inputs("question") for question in testset]

In [5]:
devset[0]

Example({'question': 'Is there support to multiple versions of the query/document embedding models to co-exist at a given time? (helps with live experiments of new model versions)'}) (input_keys={'question'})

In [6]:
metricLM = dspy.OpenAI(model = 'gpt-4o', max_tokens = 1000, model_type='chat')

class Assess(dspy.Signature):
    """Assess the quality of an answer to a question."""

    context = dspy.InputField(desc = "The context for answering the question")
    assessment_criterion = dspy.InputField(desc = "The evaluation criterion")
    assessed_answer = dspy.InputField(desc = "The answer to the question")
    assessment_answer = dspy.OutputField(desc = "A rating between 1 and 5. Only output the rating and nothing else.")

def llm_metric(gold, pred, trace = None):
    predicted_answer = pred.answer
    question = gold.question

    print(f"Test question: {question}")
    print(f"Predicted answer: {predicted_answer}")

    detail = "Is the assessed answer detailed?"
    faithful = "Is the assessed text grounded in the context? Say no if it includes significant information not in the context."
    overall = f"Please rate how well this answer addresses the question, `{question}` based on the context.\n `{predicted_answer}`"

    with dspy.context(lm = metricLM):
        context = dspy.Retrieve(k = 5)(question).passages
        detail = dspy.ChainOfThought(Assess)(context = 'N/A', assessment_criterion = detail, assessed_answer = predicted_answer)
        faithful = dspy.ChainOfThought(Assess)(context = context, assessment_criterion = faithful, assessed_answer = predicted_answer)
        overall = dspy.ChainOfThought(Assess)(context = context, assessment_criterion = overall, assessed_answer = predicted_answer)

    print(f"Faithful: {faithful.assessment_answer}")
    print(f"Detail: {detail.assessment_answer}")
    print(f"Overall: {overall.assessment_answer}")

    total = float(detail.assessment_answer) + float(faithful.assessment_answer) * 2 + float(overall.assessment_answer)

    return total / 5.0


In [7]:
test_example = dspy.Example(question = "What do cross encoders do?")
test_pred = dspy.Example(answer = "They re-rank documents.")

llm_metric(test_example, test_pred)

Test question: What do cross encoders do?
Predicted answer: They re-rank documents.
Faithful: 1
Detail: 1
Overall: 2


1.0

In [8]:
test_example = dspy.Example(question="What do cross encoders do?")
test_pred = dspy.Example(answer="They index data.")

type(llm_metric(test_example, test_pred))

Test question: What do cross encoders do?
Predicted answer: They index data.
Faithful: 1
Detail: 1
Overall: 1


float

In [9]:
metricLM.inspect_history(n=3)





Assess the quality of an answer to a question.

---

Follow the following format.

Context: The context for answering the question

Assessment Criterion: The evaluation criterion

Assessed Answer: The answer to the question

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Context:
[1] «[Cross Encoders](#cross-encoders) (collapsing the use of Large Language Models for ranking into this category as well)
1. [Metadata Rankers](#metadata-rankers)
1. [Score Rankers](#score-rankers)

## Cross Encoders
Cross Encoders are one of the most well known ranking models for content-based re-ranking. There is quite a collection of pre-trained cross encoders available on [sentence transformers](https://www.sbert.net/docs/pretrained_cross-encoders.html). We are currently envisioning interfacing cross encoders with Weaviate using the following syntax.»
[2] «Bi-Encoders 

'\n\n\nAssess the quality of an answer to a question.\n\n---\n\nFollow the following format.\n\nContext: The context for answering the question\n\nAssessment Criterion: The evaluation criterion\n\nAssessed Answer: The answer to the question\n\nReasoning: Let\'s think step by step in order to ${produce the assessment_answer}. We ...\n\nAssessment Answer: A rating between 1 and 5. Only output the rating and nothing else.\n\n---\n\nContext:\n[1] «[Cross Encoders](#cross-encoders) (collapsing the use of Large Language Models for ranking into this category as well)\n1. [Metadata Rankers](#metadata-rankers)\n1. [Score Rankers](#score-rankers)\n\n## Cross Encoders\nCross Encoders are one of the most well known ranking models for content-based re-ranking. There is quite a collection of pre-trained cross encoders available on [sentence transformers](https://www.sbert.net/docs/pretrained_cross-encoders.html). We are currently envisioning interfacing cross encoders with Weaviate using the followi

In [10]:
class GenerateAnswer(dspy.Signature):
    """Answer questions based on the context"""

    context = dspy.InputField(desc = "May contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField()

class RAG(dspy.Module):
    def __init__(self, num_passages = 3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k = num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context = context, question = question)
        return dspy.Prediction(answer = prediction.answer)

In [11]:
dspy.Predict(GenerateAnswer)(question="What are Cross Encoders?")
llm.inspect_history(n=1)




Answer questions based on the context

---

Follow the following format.

Context: May contain relevant facts
Question: ${question}
Answer: ${answer}

---

Question: What are Cross Encoders?
Answer: Context: Cross Encoders are a type of neural network architecture used in natural language processing tasks. They work by taking two input sequences (such as a pair of sentences) and processing them together to produce a single output, typically a similarity score or a classification label. This approach allows the model to consider the interactions between the two sequences in a more detailed manner compared to other architectures like Bi-Encoders, which process each sequence independently before combining their representations.

Question: What are Cross Encoders?
Answer: Cross Encoders are a type of neural network architecture used in natural language processing tasks that take two input sequences and process them together to produce a single output, allowing the model to consider the 

'\n\n\nAnswer questions based on the context\n\n---\n\nFollow the following format.\n\nContext: May contain relevant facts\nQuestion: ${question}\nAnswer: ${answer}\n\n---\n\nQuestion: What are Cross Encoders?\nAnswer:\x1b Context: Cross Encoders are a type of neural network architecture used in natural language processing tasks. They work by taking two input sequences (such as a pair of sentences) and processing them together to produce a single output, typically a similarity score or a classification label. This approach allows the model to consider the interactions between the two sequences in a more detailed manner compared to other architectures like Bi-Encoders, which process each sequence independently before combining their representations.\n\nQuestion: What are Cross Encoders?\nAnswer: Cross Encoders are a type of neural network architecture used in natural language processing tasks that take two input sequences and process them together to produce a single output, allowing th

In [12]:
dspy.ChainOfThought(GenerateAnswer)(question="What are Cross Encoders?")
llm.inspect_history(n=1)




Answer questions based on the context

---

Follow the following format.

Context: May contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Question: What are Cross Encoders?

Reasoning: Let's think step by step in order to Context: Cross Encoders are a type of model used in natural language processing tasks, particularly in tasks involving sentence pair classification. They work by taking two sentences as input and processing them together through a transformer model to produce a single output, which is typically a score indicating the relationship between the two sentences (e.g., similarity, entailment, etc.).

Question: What are Cross Encoders?

Reasoning: Let's think step by step in order to produce the answer. We need to understand the role and function of Cross Encoders in natural language processing tasks. Cross Encoders take two sentences as input and process them together throug

"\n\n\nAnswer questions based on the context\n\n---\n\nFollow the following format.\n\nContext: May contain relevant facts\n\nQuestion: ${question}\n\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\n\nAnswer: ${answer}\n\n---\n\nQuestion: What are Cross Encoders?\n\nReasoning: Let's think step by step in order to\x1b Context: Cross Encoders are a type of model used in natural language processing tasks, particularly in tasks involving sentence pair classification. They work by taking two sentences as input and processing them together through a transformer model to produce a single output, which is typically a score indicating the relationship between the two sentences (e.g., similarity, entailment, etc.).\n\nQuestion: What are Cross Encoders?\n\nReasoning: Let's think step by step in order to produce the answer. We need to understand the role and function of Cross Encoders in natural language processing tasks. Cross Encoders take two sentences as input an

In [13]:
dspy.ReAct(GenerateAnswer, tools=[dspy.settings.rm])(question="What are cross encoders?")
llm.inspect_history(n=1)




Answer questions based on the context

You will be given `context`, `question` and you will respond with `answer`.

To do this, you will interleave Thought, Action, and Observation steps.

Thought can reason about the current situation, and Action can be the following types:

(1) Search[query], which takes a search query and returns one or more potentially relevant passages from a corpus
(2) Finish[answer], which returns the final `answer` and finishes the task

---

Follow the following format.

Context: May contain relevant facts

Question: ${question}

Thought 1: next steps to take based on last observation

Action 1: always either Search[query] or, when done, Finish[<answer>], where <answer> is the answer to the question itself.

Observation 1: observations based on action

Thought 2: next steps to take based on last observation

Action 2: always either Search[query] or, when done, Finish[<answer>], where <answer> is the answer to the question itself.

Observation 2: observation

'\n\n\nAnswer questions based on the context\n\nYou will be given `context`, `question` and you will respond with `answer`.\n\nTo do this, you will interleave Thought, Action, and Observation steps.\n\nThought can reason about the current situation, and Action can be the following types:\n\n(1) Search[query], which takes a search query and returns one or more potentially relevant passages from a corpus\n(2) Finish[answer], which returns the final `answer` and finishes the task\n\n---\n\nFollow the following format.\n\nContext: May contain relevant facts\n\nQuestion: ${question}\n\nThought 1: next steps to take based on last observation\n\nAction 1: always either Search[query] or, when done, Finish[<answer>], where <answer> is the answer to the question itself.\n\nObservation 1: observations based on action\n\nThought 2: next steps to take based on last observation\n\nAction 2: always either Search[query] or, when done, Finish[<answer>], where <answer> is the answer to the question itse

In [14]:
uncompiled_rag = RAG()

In [15]:
print(uncompiled_rag("What are re-rankers in search engines?").answer)

Re-rankers in search engines are models or algorithms used to re-order search results to improve their relevance to the user's query. They take into account various features and data to provide a more accurate ranking. For example, Cross Encoders use a `(query, document)` pair to output a high precision relevance score, Metadata Rankers use symbolic features like user and document attributes to predict relevance


In [16]:
llm.inspect_history(n=1)





Answer questions based on the context

---

Follow the following format.

Context: May contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
[1] «They offer the advantage of further reasoning about the relevance of results without needing specialized training. Cross Encoders can be interfaced with Weaviate to re-rank search results, trading off performance for slower search speed. * **Metadata Rankers** are context-based re-rankers that use symbolic features to rank relevance. They take into account user and document features, such as age, gender, location, preferences, release year, genre, and box office, to predict the relevance of candidate documents. By incorporating metadata features, these rankers offer a more personalized and context-aware search experience.»
[2] «As described in our [previous article](https://weaviate.io/blog/ranking-models-for-better-search), re-ranking 

"\n\n\nAnswer questions based on the context\n\n---\n\nFollow the following format.\n\nContext: May contain relevant facts\n\nQuestion: ${question}\n\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\n\nAnswer: ${answer}\n\n---\n\nContext:\n[1] «They offer the advantage of further reasoning about the relevance of results without needing specialized training. Cross Encoders can be interfaced with Weaviate to re-rank search results, trading off performance for slower search speed. * **Metadata Rankers** are context-based re-rankers that use symbolic features to rank relevance. They take into account user and document features, such as age, gender, location, preferences, release year, genre, and box office, to predict the relevance of candidate documents. By incorporating metadata features, these rankers offer a more personalized and context-aware search experience.»\n[2] «As described in our [previous article](https://weaviate.io/blog/ranking-models-for-bette

In [17]:
from dspy.evaluate.evaluate import Evaluate

evaluate = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)
evaluate(RAG(), metric = llm_metric)

  0%|          | 0/10 [00:00<?, ?it/s]Test question: Is there support to multiple versions of the query/document embedding models to co-exist at a given time? (helps with live experiments of new model versions)
Predicted answer: Context:
[1] «These rewritten queries and prompts ensure that the search process better understands and retrieves relevant documents and the language model is prompted optimally. Query rewriting can be achieved as demonstrated below. ![rewrite](./img/image9.png)
[Query Rewriting - Ma et al. 2023](https://arxiv.org/abs/2305.142
Faithful: 1
Detail: 1
Overall: 1
Average Metric: 0.8 / 1  (80.0):  10%|█         | 1/10 [00:00<00:05,  1.79it/s]Test question: How can I retrieve the total object count in a class?
Predicted answer: You can retrieve the total object count in a class by using the Weaviate API to perform an object count query for the specific class. This ensures that the count is accurate and specific to the class in question.
Faithful: 3
Detail: 3
Overall:

,question,answer,llm_metric
0,Is there support to multiple versions of the query/document embedding models to co-exist at a given time? (helps with live experiments of new model versions),Context: [1] «These rewritten queries and prompts ensure that the search process better understands and retrieves relevant documents and the language model is prompted optimally....,✔️ [0.8]
1,How can I retrieve the total object count in a class?,You can retrieve the total object count in a class by using the Weaviate API to perform an object count query for the specific class....,✔️ [2.4]
2,How do I get the cosine similarity from Weaviate's certainty?,"The cosine similarity from Weaviate's certainty can be directly obtained as they are equivalent. Certainty is a number between 0 and 1, which corresponds to...",✔️ [2.4]
3,The quality of my search results change depending on the specified limit. Why? How can I fix this?,Context: [1] «| By re-ranking the results we are able to get the clip where Jonathan Frankle describes the benchmarks created by Ofir Press et...,✔️ [0.8]
4,Why did you use GraphQL instead of SPARQL?,"Context: [1] «GraphQL seems the perfect solution for intuitive database interaction and efficient development. Weaviate still uses traditional RESTful endpoints (using OpenAPI/Swagger) to add data,...",✔️ [2.2]


184.0

In [18]:
llm.inspect_history(n=1)




Answer questions based on the context

---

Follow the following format.

Context: May contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
[1] «Furthermore, the most popular library hnswlib only supports snapshotting, but not individual writes to disk. To get to where Weaviate is today, a custom HNSW implementation was needed. It follows the same principles [as outlined in this paper](https://arxiv.org/abs/1603.09320) but extends it with more features. Each write is added to a [write-ahead log](https://martinfowler.com/articles/patterns-of-distributed-systems/wal.html). Additionally, since inserts into HNSW are not mutable by default, Weaviate internally assigns an immutable document ID that allows for updates.»
[2] «Due to its relatively high memory footprint, HNSW is only cost-efficient in high-throughput scenarios. However, HNSW is inherently optimized for in-memory access. 

'\n\n\nAnswer questions based on the context\n\n---\n\nFollow the following format.\n\nContext: May contain relevant facts\n\nQuestion: ${question}\n\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\n\nAnswer: ${answer}\n\n---\n\nContext:\n[1] «Furthermore, the most popular library hnswlib only supports snapshotting, but not individual writes to disk. To get to where Weaviate is today, a custom HNSW implementation was needed. It follows the same principles [as outlined in this paper](https://arxiv.org/abs/1603.09320) but extends it with more features. Each write is added to a [write-ahead log](https://martinfowler.com/articles/patterns-of-distributed-systems/wal.html). Additionally, since inserts into HNSW are not mutable by default, Weaviate internally assigns an immutable document ID that allows for updates.»\n[2] «Due to its relatively high memory footprint, HNSW is only cost-efficient in high-throughput scenarios. However, HNSW is inherently optimized

In [19]:
metricLM.inspect_history(n=3)




Assess the quality of an answer to a question.

---

Follow the following format.

Context: The context for answering the question

Assessment Criterion: The evaluation criterion

Assessed Answer: The answer to the question

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Context:
[1] «Furthermore, the most popular library hnswlib only supports snapshotting, but not individual writes to disk. To get to where Weaviate is today, a custom HNSW implementation was needed. It follows the same principles [as outlined in this paper](https://arxiv.org/abs/1603.09320) but extends it with more features. Each write is added to a [write-ahead log](https://martinfowler.com/articles/patterns-of-distributed-systems/wal.html). Additionally, since inserts into HNSW are not mutable by default, Weaviate internally assigns an immutable document ID that allows for updates.

"\n\n\nAssess the quality of an answer to a question.\n\n---\n\nFollow the following format.\n\nContext: The context for answering the question\n\nAssessment Criterion: The evaluation criterion\n\nAssessed Answer: The answer to the question\n\nReasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...\n\nAssessment Answer: A rating between 1 and 5. Only output the rating and nothing else.\n\n---\n\nContext:\n[1] «Furthermore, the most popular library hnswlib only supports snapshotting, but not individual writes to disk. To get to where Weaviate is today, a custom HNSW implementation was needed. It follows the same principles [as outlined in this paper](https://arxiv.org/abs/1603.09320) but extends it with more features. Each write is added to a [write-ahead log](https://martinfowler.com/articles/patterns-of-distributed-systems/wal.html). Additionally, since inserts into HNSW are not mutable by default, Weaviate internally assigns an immutable document ID t

In [20]:
from dspy.teleprompt import BootstrapFewShot

optimizer = BootstrapFewShot(metric = llm_metric, max_labeled_demos = 8, max_rounds = 3)
compiled_rag = optimizer.compile(uncompiled_rag, trainset = trainset)

  0%|          | 0/20 [00:00<?, ?it/s]

Test question: Why would I use Weaviate as my vector database?
Predicted answer: Context:
[1] «Imagine you were using a MySQL database, but after you imported your data it would be read-only for the rest of time. That's not how it works, right? So why should it work like this for vector searching? In this article, I'm going to introduce you to Weaviate, a vector database that removes many of the limitations imposed


  5%|▌         | 1/20 [00:00<00:12,  1.56it/s]

Faithful: 1
Detail: 1
Overall: 1
Test question: What is the difference between Weaviate and for example Elasticsearch?
Predicted answer: Context:
[1] «For example, I can query Weaviate for articles related to: "urban planning in Europe", and the vector database (in the case of my demo – [Weaviate](/developers/weaviate/)) responds with a series of articles about the topic, such as "The cities designed to be capitals".<br/>
You can


 10%|█         | 2/20 [00:01<00:15,  1.17it/s]

...

Faithful: 5
Detail: 1
Overall: 1
Test question: How should I configure the size of my instance?
Predicted answer: Context:
[1] «However, before Go 1.19, you only had a single knob to turn: the GOGC environment variable. This variable accepted a relative target compared to the current live heap size. The default value for GOGC is 100, meaning that the heap should double (i.e. grow by 100 percent) before GC should


 20%|██        | 4/20 [00:03<00:12,  1.33it/s]


Faithful: 3
Detail: 1
Overall: 1


  0%|          | 0/20 [00:00<?, ?it/s]

Bootstrapped 4 full traces after 1 examples in round 2.


In [21]:
compiled_rag("What do cross encoders do?").answer

'Context: [1] «[Cross Encoders](#cross-encoders) (collapsing the use of Large Language Models for ranking into this category as well) 1. [Metadata Rankers](#metadata-rankers) 1. [Score Rankers](#score-rankers) ## Cross Encoders Cross Encoders are one of the most'

In [22]:
llm.inspect_history(n=1)





Answer questions based on the context

---

Follow the following format.

Context: May contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
[1] «Imagine you were using a MySQL database, but after you imported your data it would be read-only for the rest of time. That's not how it works, right? So why should it work like this for vector searching? In this article, I'm going to introduce you to Weaviate, a vector database that removes many of the limitations imposed by ANN libraries. ## Overview
In this article we will cover:

* how ANN models enable fast & large-scale vector searches
* where popular ANN libraries fall short
* what Weaviate is and how it can bring your vector search needs to production
* a glimpse of how Weaviate works under the hood

## What is Weaviate?»
[2] «However, consistent with Weaviate's commitment to creating truly open-source software, customers using t

'\n\n\nAnswer questions based on the context\n\n---\n\nFollow the following format.\n\nContext: May contain relevant facts\n\nQuestion: ${question}\n\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\n\nAnswer: ${answer}\n\n---\n\nContext:\n[1] «Imagine you were using a MySQL database, but after you imported your data it would be read-only for the rest of time. That\'s not how it works, right? So why should it work like this for vector searching? In this article, I\'m going to introduce you to Weaviate, a vector database that removes many of the limitations imposed by ANN libraries. ## Overview\nIn this article we will cover:\n\n* how ANN models enable fast & large-scale vector searches\n* where popular ANN libraries fall short\n* what Weaviate is and how it can bring your vector search needs to production\n* a glimpse of how Weaviate works under the hood\n\n## What is Weaviate?»\n[2] «However, consistent with Weaviate\'s commitment to creating truly open-

In [23]:
evaluate(compiled_rag, metric=llm_metric)

  0%|          | 0/10 [00:00<?, ?it/s]Test question: Is there support to multiple versions of the query/document embedding models to co-exist at a given time? (helps with live experiments of new model versions)
Predicted answer: Context: [1] «These rewritten queries and prompts ensure that the search process better understands and retrieves relevant documents and the language model is prompted optimally. Query rewriting can be achieved as demonstrated below. ![rewrite](./img/image9.png) [Query Rewriting - Ma et al. 2023](https://arxiv.org/abs/2305.142
Faithful: 1
Detail: 1
Overall: 1
Average Metric: 0.8 / 1  (80.0):  10%|█         | 1/10 [00:00<00:06,  1.48it/s]Test question: How can I retrieve the total object count in a class?
Predicted answer: To retrieve the total object count in a class, you can perform an object count operation after importing the objects. This can be done using the appropriate API or client library methods provided by Weaviate. The exact method may vary dependin

,question,answer,llm_metric
0,Is there support to multiple versions of the query/document embedding models to co-exist at a given time? (helps with live experiments of new model versions),Context: [1] «These rewritten queries and prompts ensure that the search process better understands and retrieves relevant documents and the language model is prompted optimally....,✔️ [0.8]
1,How can I retrieve the total object count in a class?,"To retrieve the total object count in a class, you can perform an object count operation after importing the objects. This can be done using...",✔️ [1.6]
2,How do I get the cosine similarity from Weaviate's certainty?,"To get the cosine similarity from Weaviate's certainty, you can use the certainty value directly as it is designed to work with cosine",✔️ [2.0]
3,The quality of my search results change depending on the specified limit. Why? How can I fix this?,Context: [1] «| By re-ranking the results we are able to get the clip where Jonathan Frankle describes the benchmarks created by Ofir Press et...,✔️ [0.8]
4,Why did you use GraphQL instead of SPARQL?,"Context: [1] «GraphQL seems the perfect solution for intuitive database interaction and efficient development. Weaviate still uses traditional RESTful endpoints (using OpenAPI/Swagger) to add data,...",✔️ [2.2]


188.0

In [25]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

optimizer = BootstrapFewShotWithRandomSearch(
    metric = llm_metric,
    max_bootstrapped_demos=4,
    max_labeled_demos=4,
    max_rounds=1,
    num_candidate_programs=2,
    num_threads=2
)

second_compiled_rag = optimizer.compile(RAG(), trainset=trainset)

Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 2 candidate sets.


  0%|          | 0/20 [00:00<?, ?it/s]

Test question: Why would I use Weaviate as my vector database?
Predicted answer: Context:
[1] «Imagine you were using a MySQL database, but after you imported your data it would be read-only for the rest of time. That's not how it works, right? So why should it work like this for vector searching? In this article, I'm going to introduce you to Weaviate, a vector database that removes many of the limitations imposed
Test question: What is the difference between Weaviate and for example Elasticsearch?
Predicted answer: Context:
[1] «For example, I can query Weaviate for articles related to: "urban planning in Europe", and the vector database (in the case of my demo – [Weaviate](/developers/weaviate/)) responds with a series of articles about the topic, such as "The cities designed to be capitals".<br/>
You can


Average Metric: 0.8 / 1  (80.0):   5%|▌         | 1/20 [00:00<00:10,  1.81it/s]

Faithful: 1
Detail: 1
Overall: 1


...

Backing off 1.1 seconds after 2 tries calling function <function GPT3.request at 0x10f134860> with kwargs {}


Average Metric: 17.8 / 19  (93.7):  95%|█████████▌| 19/20 [03:13<00:09,  9.81s/it]

Faithful: 2
Detail: 1
Overall: 1


ERROR:backoff:Giving up request(...) after 3 tries (openai.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-vhY7zZIHWbJr60TUHz326zaG on tokens per min (TPM): Limit 30000, Used 28451, Requested 2554. Please try again in 2.01s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:dspy.evaluate.evaluate:2024-09-12T19:13:53.070850Z [error    ] Error for example in dev set: 		 Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-vhY7zZIHWbJr60TUHz326zaG on tokens per min (TPM): Limit 30000, Used 28451, Requested 2554. Please try again in 2.01s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}} [dspy.evaluate.evaluate] filename=evaluate.py lineno=183
Average Metric: 17.8 / 20  (89.0): 100%|██████████| 20/20 [03:14<00:00,  9.

Score: 89.0 for set: [4]
Scores so far: [177.0, 177.0, 88.0, 114.0, 89.0]
Best score: 177.0
Average of max per entry across top 1 scores: 1.77
Average of max per entry across top 2 scores: 1.77
Average of max per entry across top 3 scores: 1.9
Average of max per entry across top 5 scores: 1.97
Average of max per entry across top 8 scores: 1.97
Average of max per entry across top 9999 scores: 1.97
5 candidate programs found.


In [26]:
second_compiled_rag("What do cross encoders do?")

Prediction(
    answer='Cross Encoders are used for content-based re-ranking by using a classification mechanism for data pairs to calculate a similarity score between 0 and 1, indicating the similarity between the items in the'
)

In [27]:
llm.inspect_history(n=1)




Answer questions based on the context

---

Follow the following format.

Context: May contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
[1] «[Cross Encoders](#cross-encoders) (collapsing the use of Large Language Models for ranking into this category as well)
1. [Metadata Rankers](#metadata-rankers)
1. [Score Rankers](#score-rankers)

## Cross Encoders
Cross Encoders are one of the most well known ranking models for content-based re-ranking. There is quite a collection of pre-trained cross encoders available on [sentence transformers](https://www.sbert.net/docs/pretrained_cross-encoders.html). We are currently envisioning interfacing cross encoders with Weaviate using the following syntax.»
[2] «Bi-Encoders are fast, but are not as accurate as the expensive fisherman aka the Cross-Encoder. Cross-Encoders are time-consuming, like the fisherman who would need to limit the numb

'\n\n\nAnswer questions based on the context\n\n---\n\nFollow the following format.\n\nContext: May contain relevant facts\n\nQuestion: ${question}\n\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\n\nAnswer: ${answer}\n\n---\n\nContext:\n[1] «[Cross Encoders](#cross-encoders) (collapsing the use of Large Language Models for ranking into this category as well)\n1. [Metadata Rankers](#metadata-rankers)\n1. [Score Rankers](#score-rankers)\n\n## Cross Encoders\nCross Encoders are one of the most well known ranking models for content-based re-ranking. There is quite a collection of pre-trained cross encoders available on [sentence transformers](https://www.sbert.net/docs/pretrained_cross-encoders.html). We are currently envisioning interfacing cross encoders with Weaviate using the following syntax.»\n[2] «Bi-Encoders are fast, but are not as accurate as the expensive fisherman aka the Cross-Encoder. Cross-Encoders are time-consuming, like the fisherman who 

In [28]:
evaluate(second_compiled_rag, metric=llm_metric)

  0%|          | 0/10 [00:00<?, ?it/s]Test question: Is there support to multiple versions of the query/document embedding models to co-exist at a given time? (helps with live experiments of new model versions)
Predicted answer: Context:
[1] «These rewritten queries and prompts ensure that the search process better understands and retrieves relevant documents and the language model is prompted optimally. Query rewriting can be achieved as demonstrated below. ![rewrite](./img/image9.png)
[Query Rewriting - Ma et al. 2023](https://arxiv.org/abs/2305.142
Faithful: 1
Detail: 1
Overall: 1
Average Metric: 0.8 / 1  (80.0):  10%|█         | 1/10 [00:00<00:06,  1.41it/s]Test question: How can I retrieve the total object count in a class?
Predicted answer: You can retrieve the total object count in a class by using the Weaviate API to perform an object count query for the specific class. This ensures that the count is accurate and specific to the class in question.
Faithful: 3
Detail: 3
Overall:

,question,answer,llm_metric
0,Is there support to multiple versions of the query/document embedding models to co-exist at a given time? (helps with live experiments of new model versions),Context: [1] «These rewritten queries and prompts ensure that the search process better understands and retrieves relevant documents and the language model is prompted optimally....,✔️ [0.8]
1,How can I retrieve the total object count in a class?,You can retrieve the total object count in a class by using the Weaviate API to perform an object count query for the specific class....,✔️ [2.4]
2,How do I get the cosine similarity from Weaviate's certainty?,"The cosine similarity from Weaviate's certainty can be directly obtained as they are equivalent. Certainty is a number between 0 and 1, which corresponds to...",✔️ [2.4]
3,The quality of my search results change depending on the specified limit. Why? How can I fix this?,Context: [1] «| By re-ranking the results we are able to get the clip where Jonathan Frankle describes the benchmarks created by Ofir Press et...,✔️ [0.8]
4,Why did you use GraphQL instead of SPARQL?,"Context: [1] «GraphQL seems the perfect solution for intuitive database interaction and efficient development. Weaviate still uses traditional RESTful endpoints (using OpenAPI/Swagger) to add data,...",✔️ [2.2]


184.0

In [33]:
from dspy.teleprompt import MIPROv2

llm_prompter = dspy.OpenAI(model = 'gpt-4o', max_tokens = 2000, model_type='chat')

optimizer = MIPROv2(
    task_model=dspy.settings.lm,
    metric = llm_metric,
    prompt_model=llm_prompter,
    num_candidates=5
)

kwargs = dict(num_threads=1, display_progress=True, display_table=0)
third_compiled_rag = optimizer.compile(RAG(), trainset=trainset, valset=devset, max_bootstrapped_demos=4, max_labeled_demos=4, eval_kwargs=kwargs)


Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 5 * 1 lm calls in program + (2) lm calls in program aware proposer = 17 prompt model calls
- Task Model: 25 examples in minibatch * 30 batches + 20 examples in train set * 3 full evals = 810 task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may co